<a href="https://colab.research.google.com/github/carloslme/automating-boring-stuff/blob/main/Chapter_11_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web scraping is the term for using a program to download and process content from the Web.

In this chapter, you will learn about several modules that make it easy to scrape web pages in Python. 
* webbrowser . Comes with Python and opens a browser to a specific page. 
* Requests . Downloads files and web pages from the Internet. 
* Beautiful Soup . Parses HTML, the format that web pages are written in. 
* Selenium . Launches and controls a web browser. Selenium is able to fill in forms and simulate mouse clicks in this browser.

In [1]:
!pip install requests

In [2]:
import requests

In [3]:
res = requests.get('https://automatetheboringstuff.com/files/rj.txt')

In [4]:
type(res)

requests.models.Response

In [5]:
res.status_code == requests.codes.ok

True

In [6]:
len(res.text)

178978

In [7]:
print(res.text[:250])

The Project Gutenberg EBook of Romeo and Juliet, by William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Projec
